Me gustaría que el usuario me de:
- Pairings: uno o más combinaciones de alimentos con los que quiere maridar el vino. (OK)
- Body, tannis, sweetness, acidity: de 0 a 1 (OK)
- Uvas: una selección de uvas que yo determine + otras (todas las que no estén incluidas dentro de las que le doy como opción individual). (WIP)
- Price: el rango de precios en el cual buscar vinos. (WIP)

In [103]:
import random
import pandas as pd
import numpy as np

In [101]:
# randint: https://www.w3schools.com/python/ref_random_randint.asp
# Pandas .quantile: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html
# np.random.normal: https://numpy.org/doc/2.1/reference/random/generated/numpy.random.normal.html
# np.clip: https://numpy.org/doc/2.1/reference/generated/numpy.clip.html 


# Selección de pairing
def select_random_meal(meal_df):
    i = np.random.randint(0, len(meal_df))
    return meal_df.loc[i]

# Elección de categoría según un valor dado
def get_category(value, quantiles=[.25, .5, .75], categories=["leve", "moderado", "marcado", "intenso"]):
    if len(quantiles) + 1 != len(categories):
        raise IndexError("Length of quantiles and categories does not match!")
    
    for i, q in enumerate(quantiles):
        if value <= q:
            return categories[i]
        
    return categories[-1]

# Creación de perfiles de sabor por pairing (para elecciones que hagan sentido con la elección del user)
def build_pairing_profile(df, pairing_cols, quantiles=[.25, .5, .75]):
    profiles = {}
    deviations = {}
    for pairing in pairing_cols:
        pairing_subset = df[df[pairing] == 1]
        profiles[pairing] = {}
        deviations[pairing] = {}
        for taste in ["body", "tannis", "sweetness", "acidity"]:
            # Cuantiles
            cuartiles = pairing_subset[taste].quantile(quantiles)
            profiles[pairing][taste] = cuartiles
            # Desvío Estandar (para distribución normal en select_taste_profile)
            std = pairing_subset[taste].std()
            deviations[pairing][taste] = std

    return profiles, deviations

# Selección del perfil de vino con algo de sentido común (en general)
def select_taste_profile(pairing, profiles, deviations, prob=.1, categories=["leve", "moderado", "marcado", "intenso"]):
    selected_profile = {}
    for taste in ["body", "tannis", "sweetness", "acidity"]:
        if random.random() < prob:
            selected_profile[taste] = str(np.random.choice(categories))
        else:
            median = profiles[pairing][taste][.5]
            std = deviations[pairing][taste]
            valor_seleccionado = np.random.normal(loc=median, scale=std)
            selected_profile[taste] = get_category(valor_seleccionado, categories=categories)
    return selected_profile


In [ ]:
wines_df = pd.read_csv("../../src/data/transformed/wines_clean.csv")
pairings = pd.read_csv("../../src/data/processed/aux/pairings.csv")
pairings = list(pairings["pairings"])
meals_df = pd.read_excel("../../src/data/raw/meals/Meals.xlsx")
scaler = MinMaxScaler()

taste_columns = ["body", "tannis", "sweetness", "acidity"]

wines_copy = wines_df.copy()
wines_copy[taste_columns] = scaler.fit_transform(wines_df[taste_columns])


# Creación de perfiles de sabor por pairing y obtención de sus desvíos para selección según distribución normal
profiles, deviations = build_pairing_profile(df=wines_copy, pairing_cols=pairings, quantiles=[.25, .5, .75])

# Elección aleatoria de una meal y obtención de su pairing principal
selected_meal = select_random_meal(meals_df)
main_pairing = selected_meal["Ingrediente Principal"].lower()

# Elección del perfil del vino, teniendo en cuenta algo de sentido común de los usuarios
selected_profile = select_taste_profile(pairing=main_pairing, profiles=profiles, deviations=deviations, prob=.1)



{'body': 'marcado',
 'tannis': 'marcado',
 'sweetness': 'moderado',
 'acidity': 'marcado'}